In [ ]:
#| default_exp date_compare

# date_compare
> find corresponding dates from location history and image exif data

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from bisect import bisect
from collections import namedtuple
import pathlib

from regps.json_explorer import *
from regps.exif_explorer import *
from GPSPhoto import gpsphoto

Use exif explorer to build up a list of image metadata

In [ ]:
import glob
path = 'sample-data/*.jpg'
images = glob.glob(path)
len(images)

2

In [ ]:
image_list = extract_exif(images)
assert len(image_list) == 2

In [ ]:
image_list[0].exif.get("datetime_original")
# note that some of these can be None, since some images are created by google photos

'2018:11:30 07:57:21'

Now we need to convert the to a timestamp the dates from exif

In [ ]:
#| export
from datetime import datetime
def to_timestamp(date):
    date_format = "%Y:%m:%d %H:%M:%S"
    timestamp = datetime.strptime(date, date_format)
    return int(timestamp.timestamp())

In [ ]:
# testing to make sure we get the right value back
timestamp = to_timestamp(image_list[0].exif.get("datetime_original"))
timestamp

1543579041

In [ ]:
timestamp2 = to_timestamp(image_list[-1].exif.get("datetime_original"))
timestamp2

1543547561

In [ ]:
#| export

def delta_to_minutes(delta):
    return int(delta.total_seconds() // 60)

def get_time_delta(a, b):
    a = datetime.fromtimestamp(a)
    b = datetime.fromtimestamp(b)
    if a > b:
        return delta_to_minutes(a - b)
    return delta_to_minutes(b - a)

In [ ]:
results = get_time_delta(timestamp2, timestamp)
results

524

In [ ]:
# quick test
assert(get_time_delta(timestamp2, timestamp)) == 524

Now lets pull some information from location history to compare

In [ ]:
file_to_open = "sample-data/sample.json"

locations = get_locations(file_to_open)
assert len(locations) == 58

In [ ]:
locations_w_gps = build_location_history(locations)
assert len(locations_w_gps) == 58

In [ ]:
locations_w_gps[0]

Location(timestamp=1467216494, latitude=446549411, longitude=-635836042, accuracy=41, altitude=0)

Now lets come up with a bruteforce solution to finding the aligning dates from both datasets

In [ ]:
#| export

# optimized solution
def get_smallest_deltas(image_list, locations):
    d = {}
    location_timestamps = [l.timestamp for l in locations]
    for image_index, image in enumerate(image_list):
        image_timestamp = to_timestamp(image.exif.get("datetime_original"))
        
        index = bisect(location_timestamps, image_timestamp)
        if index == len(location_timestamps):
            index = index -1
        delta = get_time_delta(image_timestamp, locations[index].timestamp) 
        d[image_index] = index
    return d
    

In [ ]:
d = get_smallest_deltas(image_list, locations_w_gps)  
d

{0: 57, 1: 57}

Now we can take a peek at how close the date ranges between the image timestamps and location history timestamps are.

In [ ]:
#| export
# de-google lat/long

# new data structure to hold images w gps metadata
ImageGPS = namedtuple("ImageGPS", ["image_path", "gps", "alt"])

def convert_to_decimal(lat, long):
    # 1e7 is the value to divide by to convert from latitudeE7/longitudeE7 fields
    return lat/1e7, long/1e7


def de_google_gps_info(d, image_list, locations):
    imgs_w_data = []
    for image_index, location_index in d.items():
        image_time = to_timestamp(image_list[image_index].exif.get("datetime_original"))
        location_time = locations[location_index].timestamp
        delta = get_time_delta(image_time, location_time)
        lat, long = convert_to_decimal(locations[location_index].latitude, locations[location_index].longitude)
        alt = locations[location_index].altitude
        imgs_w_data.append(ImageGPS(image_list[image_index].image_path, (lat, long), alt))
    return imgs_w_data

In [ ]:
imgs_w_data = de_google_gps_info(d, image_list, locations_w_gps)
imgs_w_data[0]

ImageGPS(image_path='sample-data/sample.jpg', gps=(44.6551683, -63.5835479), alt=0)

From here we can start associating the GPS data from the location history to the images that Google Photo's has stripped.

In [ ]:
#| export

def write_gps_info_to_images(image_list, output_path):
    for image in image_list:
        info = gpsphoto.GPSInfo(image.gps, alt=image.alt)
        photo = gpsphoto.GPSPhoto(image.image_path)
        filename = pathlib.Path(image.image_path).name
        photo.modGPSData(info, f'{output_path}/{filename}')

In [ ]:
# finally write out new files with GPS data merged in exif
output_path = '/tmp'
write_gps_info_to_images(imgs_w_data, output_path)

# EXIF GPS fields for reference

```
exif:GPSAltitude=94940/11161
exif:GPSAltitudeRef=.
exif:GPSDestBearing=227653/2182
exif:GPSDestBearingRef=T
exif:GPSHPositioningError=33479/4096
exif:GPSImgDirection=227653/2182
exif:GPSImgDirectionRef=T
exif:GPSInfo=2272
exif:GPSLatitude=45/1, 30/1, 5110/100
exif:GPSLatitudeRef=N
exif:GPSLongitude=73/1, 31/1, 3981/100
exif:GPSLongitudeRef=W
exif:GPSSpeed=4744/18627
exif:GPSSpeedRef=K
```

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export
nbdev_export()